In [19]:
from datetime import datetime, timedelta
import requests

# Token de autenticação
auth_token = "6amfE0YN0EGbx"

# Obter a data de ontem
ontem = datetime.now() - timedelta(days=1)
data_formatada = ontem.strftime("%Y-%m-%d")

# Lista de zonas
zonas = ["DE", "IT", "PT"]

# Iterar sobre as zonas
for zona in zonas:
    # URL da solicitação
    url = f"https://api.electricitymap.org/v3/carbon-intensity/history?zone={zona}&start={data_formatada}&end={data_formatada}"

    # Fazendo a solicitação
    response = requests.get(url, headers={"auth-token": auth_token})

    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        # Verificando se há dados de intensidade de carbono disponíveis
        if "history" in data and data["history"]:
            print(f"Intensidade de carbono para {zona} no dia {data_formatada}:")
            # Iterar sobre o histórico de intensidade de carbono
            for entry in data["history"]:
                carbon_intensity = entry["carbonIntensity"]
                datetime_str = entry["datetime"]
                print(f"- {datetime_str} - Intensidade de carbono: {carbon_intensity} gCO2eq/kWh")
        else:
            print(f"Não há dados disponíveis de intensidade de carbono para {zona} no dia {data_formatada}")
    else:
        print(f"Erro ao fazer a solicitação para {zona}: {response.status_code}")

Intensidade de carbono para DE no dia 2024-05-16:
- 2024-05-16T17:00:00.000Z - Intensidade de carbono: 323 gCO2eq/kWh
- 2024-05-16T18:00:00.000Z - Intensidade de carbono: 338 gCO2eq/kWh
- 2024-05-16T19:00:00.000Z - Intensidade de carbono: 340 gCO2eq/kWh
- 2024-05-16T20:00:00.000Z - Intensidade de carbono: 338 gCO2eq/kWh
- 2024-05-16T21:00:00.000Z - Intensidade de carbono: 314 gCO2eq/kWh
- 2024-05-16T22:00:00.000Z - Intensidade de carbono: 291 gCO2eq/kWh
- 2024-05-16T23:00:00.000Z - Intensidade de carbono: 277 gCO2eq/kWh
- 2024-05-17T00:00:00.000Z - Intensidade de carbono: 261 gCO2eq/kWh
- 2024-05-17T01:00:00.000Z - Intensidade de carbono: 256 gCO2eq/kWh
- 2024-05-17T02:00:00.000Z - Intensidade de carbono: 248 gCO2eq/kWh
- 2024-05-17T03:00:00.000Z - Intensidade de carbono: 250 gCO2eq/kWh
- 2024-05-17T04:00:00.000Z - Intensidade de carbono: 276 gCO2eq/kWh
- 2024-05-17T05:00:00.000Z - Intensidade de carbono: 280 gCO2eq/kWh
- 2024-05-17T06:00:00.000Z - Intensidade de carbono: 252 gCO2eq/kW

In [3]:
import numpy as np
import requests

# Função para as intensidades de carbono para uma zona específica no dia anterior
def buscar_intensidades_carbono(zone):
    url = "https://api.electricitymap.org/v3/carbon-intensity/history"
    params = {
        "zone": zone
    }
    headers = {
        "auth-token": "YOUR_API_TOKEN"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        intensidades_carbono = [entry["carbonIntensity"] for entry in data["history"]]
        return intensidades_carbono[-24:]  # Retorna apenas as últimas 24 horas
    else:
        print("Falha ao buscar as intensidades de carbono.")
        return None

# Intensidades de carbono para cada zona no dia anterior
intensidades_carbono_de = buscar_intensidades_carbono("DE")
intensidades_carbono_it = buscar_intensidades_carbono("IT")
intensidades_carbono_pt = buscar_intensidades_carbono("PT")

# Se algum dos valores for None, significa que falhou
if None in (intensidades_carbono_de, intensidades_carbono_it, intensidades_carbono_pt):
    print("Falha ao buscar as intensidades de carbono para uma ou mais zonas.")
    exit()

# Valores das emissões de carbono para cada zona
print("Intensidades de carbono para Alemanha:", intensidades_carbono_de)
print("Intensidades de carbono para Itália:", intensidades_carbono_it)
print("Intensidades de carbono para Portugal:", intensidades_carbono_pt)
print()

# Exemplo de dados de carregamento para três empresas (24 horas)
carregamento_empresa_1 = np.array([5, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4])
carregamento_empresa_2 = np.array([4, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3])
carregamento_empresa_3 = np.array([3, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2])

# Função para calcular as emissões totais diárias
def calcular_emissoes_diarias(carregamento, emissoes):
    emissoes_horarias = carregamento * emissoes
    emissoes_diarias = np.sum(emissoes_horarias)
    return emissoes_diarias

# Função para calcular os cenários de emissões
def calcular_cenarios(carregamento_total, emissoes):
    emissoes_ordenadas_crescente = np.sort(emissoes)
    emissoes_ordenadas_decrescente = np.sort(emissoes)[::-1]
    
    capacidade_por_hora = 10  # Definir capacidade por hora como 10
    horas_necessarias = int(np.ceil(carregamento_total / capacidade_por_hora))
    
    best_case = capacidade_por_hora * emissoes_ordenadas_crescente[:horas_necessarias]
    worst_case = capacidade_por_hora * emissoes_ordenadas_decrescente[:horas_necessarias]
    
    best_case_total = np.sum(best_case)
    worst_case_total = np.sum(worst_case)
    
    return best_case_total, worst_case_total

# Calcular o carregamento total para cada empresa
carregamento_empresa_1_total = np.sum(carregamento_empresa_1)
carregamento_empresa_2_total = np.sum(carregamento_empresa_2)
carregamento_empresa_3_total = np.sum(carregamento_empresa_3)

# Cálculo das emissões reais, best case e worst case para cada empresa
emissoes_empresa_1 = calcular_emissoes_diarias(carregamento_empresa_1, intensidades_carbono_de)
emissoes_empresa_2 = calcular_emissoes_diarias(carregamento_empresa_2, intensidades_carbono_it)
emissoes_empresa_3 = calcular_emissoes_diarias(carregamento_empresa_3, intensidades_carbono_pt)

best_case_1, worst_case_1 = calcular_cenarios(carregamento_empresa_1_total, intensidades_carbono_de)
best_case_2, worst_case_2 = calcular_cenarios(carregamento_empresa_2_total, intensidades_carbono_it)
best_case_3, worst_case_3 = calcular_cenarios(carregamento_empresa_3_total, intensidades_carbono_pt)

# Calcular a distância ao best case e ao worst case
def calcular_distancia_best_worst(emissoes_reais, best_case, worst_case):
    distancia_best = emissoes_reais / best_case
    distancia_worst = emissoes_reais / worst_case
    
    print(emissoes_reais, best_case, worst_case)
    print(distancia_best, distancia_worst)
    return distancia_best, distancia_worst

distancia_best_1, distancia_worst_1 = calcular_distancia_best_worst(emissoes_empresa_1, best_case_1, worst_case_1)
distancia_best_2, distancia_worst_2 = calcular_distancia_best_worst(emissoes_empresa_2, best_case_2, worst_case_2)
distancia_best_3, distancia_worst_3 = calcular_distancia_best_worst(emissoes_empresa_3, best_case_3, worst_case_3)

# Resultados
empresas = ['Empresa 1', 'Empresa 2', 'Empresa 3']
distancias_best = [distancia_best_1, distancia_best_2, distancia_best_3]
distancias_worst = [distancia_worst_1, distancia_worst_2, distancia_worst_3]

# Rank das empresas baseado na distância ao best case
ranking_best = sorted(zip(empresas, distancias_best), key=lambda x: x[1])

print("\nRanking das empresas baseado na distância ao best case de emissões de CO2:")
for i, (empresa, distancia) in enumerate(ranking_best):
    print(f"{i + 1}. {empresa}: {distancia * 100:.2f}% do best case scenario")

# Rank das empresas baseado na distância ao worst case
ranking_worst = sorted(zip(empresas, distancias_worst), key=lambda x: x[1])

print("\nRanking das empresas baseado na distância ao worst case de emissões de CO2:")
for i, (empresa, distancia) in enumerate(ranking_worst):
    print(f"{i + 1}. {empresa}: {distancia * 100:.2f}% do worst case scenario")


Intensidades de carbono para Alemanha: [155, 155, 156, 162, 177, 216, 286, 350, 374, 373, 349, 310, 293, 280, 281, 283, 280, 294, 321, 309, 291, 250, 249, 227]
Intensidades de carbono para Itália: [185, 186, 193, 198, 211, 246, 277, 289, 296, 299, 282, 269, 250, 248, 239, 235, 219, 210, 217, 217, 210, 211, 209, 210]
Intensidades de carbono para Portugal: [77, 79, 77, 75, 75, 73, 69, 74, 85, 87, 90, 89, 84, 78, 93, 89, 87, 93, 98, 82, 85, 94, 87, 73]

47038 40350 51730
1.1657496902106568 0.9092982795283201
34732 31500 37940
1.1026031746031746 0.9154454401686875
10191 10010 11610
1.018081918081918 0.8777777777777778

Ranking das empresas baseado na distância ao best case de emissões de CO2:
1. Empresa 3: 101.81% do best case scenario
2. Empresa 2: 110.26% do best case scenario
3. Empresa 1: 116.57% do best case scenario

Ranking das empresas baseado na distância ao worst case de emissões de CO2:
1. Empresa 3: 87.78% do worst case scenario
2. Empresa 1: 90.93% do worst case scenario
3. Em

In [8]:
import numpy as np
import requests

# Função para as intensidades de carbono para uma zona específica no dia anterior
def buscar_intensidades_carbono(zone):
    url = "https://api.electricitymap.org/v3/carbon-intensity/history"
    params = {
        "zone": zone
    }
    headers = {
        "auth-token": "YOUR_API_TOKEN"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        intensidades_carbono = [entry["carbonIntensity"] for entry in data["history"]]
        return intensidades_carbono[-24:]  # Retorna apenas as últimas 24 horas
    else:
        print("Falha ao buscar as intensidades de carbono.")
        return None

# Intensidades de carbono para cada zona no dia anterior
intensidades_carbono_de = buscar_intensidades_carbono("DE")
intensidades_carbono_it = buscar_intensidades_carbono("IT")
intensidades_carbono_pt = buscar_intensidades_carbono("PT")

# Se algum dos valores for None, significa que falhou
if None in (intensidades_carbono_de, intensidades_carbono_it, intensidades_carbono_pt):
    print("Falha ao buscar as intensidades de carbono para uma ou mais zonas.")
    exit()

# Valores das emissões de carbono para cada zona
print("Intensidades de carbono para Alemanha:", intensidades_carbono_de)
print("Intensidades de carbono para Itália:", intensidades_carbono_it)
print("Intensidades de carbono para Portugal:", intensidades_carbono_pt)
print()

# Exemplo de dados de carregamento para três empresas (24 horas)
carregamento_empresa_1 = np.array([5, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4])
carregamento_empresa_2 = np.array([4, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3])
carregamento_empresa_3 = np.array([3, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2])

# Função para calcular as emissões totais diárias
def calcular_emissoes_diarias(carregamento, emissoes):
    emissoes_horarias = carregamento * emissoes
    emissoes_diarias = np.sum(emissoes_horarias)
    return emissoes_diarias

# Função para calcular os cenários de emissões
def calcular_cenarios(carregamento_total, emissoes):
    emissoes_ordenadas_crescente = np.sort(emissoes)
    emissoes_ordenadas_decrescente = np.sort(emissoes)[::-1]
    
    capacidade_por_hora = 10  # Definir capacidade por hora como 10
    horas_necessarias = int(np.ceil(carregamento_total / capacidade_por_hora))
    
    best_case = capacidade_por_hora * emissoes_ordenadas_crescente[:horas_necessarias]
    worst_case = capacidade_por_hora * emissoes_ordenadas_decrescente[:horas_necessarias]
    
    best_case_total = np.sum(best_case)
    worst_case_total = np.sum(worst_case)
    
    return best_case_total, worst_case_total

# Calcular o carregamento total para cada empresa
carregamento_empresa_1_total = np.sum(carregamento_empresa_1)
carregamento_empresa_2_total = np.sum(carregamento_empresa_2)
carregamento_empresa_3_total = np.sum(carregamento_empresa_3)

# Cálculo das emissões reais, best case e worst case para cada empresa
emissoes_empresa_1 = calcular_emissoes_diarias(carregamento_empresa_1, intensidades_carbono_de)
emissoes_empresa_2 = calcular_emissoes_diarias(carregamento_empresa_2, intensidades_carbono_it)
emissoes_empresa_3 = calcular_emissoes_diarias(carregamento_empresa_3, intensidades_carbono_pt)

best_case_1, worst_case_1 = calcular_cenarios(carregamento_empresa_1_total, intensidades_carbono_de)
best_case_2, worst_case_2 = calcular_cenarios(carregamento_empresa_2_total, intensidades_carbono_it)
best_case_3, worst_case_3 = calcular_cenarios(carregamento_empresa_3_total, intensidades_carbono_pt)

# Calcular a distância ao best case e ao worst case
def calcular_distancia_best_worst(emissoes_reais, best_case, worst_case):
    distancia_best = emissoes_reais / best_case
    distancia_worst = emissoes_reais / worst_case
    return distancia_best, distancia_worst

distancia_best_1, distancia_worst_1 = calcular_distancia_best_worst(emissoes_empresa_1, best_case_1, worst_case_1)
distancia_best_2, distancia_worst_2 = calcular_distancia_best_worst(emissoes_empresa_2, best_case_2, worst_case_2)
distancia_best_3, distancia_worst_3 = calcular_distancia_best_worst(emissoes_empresa_3, best_case_3, worst_case_3)

# Normalizar as distâncias ao best case e worst case
min_dist_best = min(distancia_best_1, distancia_best_2, distancia_best_3)
max_dist_best = max(distancia_best_1, distancia_best_2, distancia_best_3)

norm_dist_best_1 = (distancia_best_1 - min_dist_best) / (max_dist_best - min_dist_best)
norm_dist_best_2 = (distancia_best_2 - min_dist_best) / (max_dist_best - min_dist_best)
norm_dist_best_3 = (distancia_best_3 - min_dist_best) / (max_dist_best - min_dist_best)

min_dist_worst = min(distancia_worst_1, distancia_worst_2, distancia_worst_3)
max_dist_worst = max(distancia_worst_1, distancia_worst_2, distancia_worst_3)

norm_dist_worst_1 = (distancia_worst_1 - min_dist_worst) / (max_dist_worst - min_dist_worst)
norm_dist_worst_2 = (distancia_worst_2 - min_dist_worst) / (max_dist_worst - min_dist_worst)
norm_dist_worst_3 = (distancia_worst_3 - min_dist_worst) / (max_dist_worst - min_dist_worst)

# Calcular o score geral ajustado
score_1 = (1 - norm_dist_best_1) + (1 - norm_dist_worst_1)
score_2 = (1 - norm_dist_best_2) + (1 - norm_dist_worst_2)
score_3 = (1 - norm_dist_best_3) + (1 - norm_dist_worst_3)

# Rank das empresas baseado no score ajustado
empresas = ['Empresa 1', 'Empresa 2', 'Empresa 3']
scores = [score_1, score_2, score_3]
ranking = sorted(zip(empresas, scores), key=lambda x: x[1], reverse=True)

print("\nRanking geral das empresas:")
for i, (empresa, score) in enumerate(ranking):
    print(f"{i + 1}. {empresa}: Score {score:.2f}")


Intensidades de carbono para Alemanha: [234, 281, 331, 407, 459, 486, 488, 462, 421, 397, 385, 382, 391, 426, 447, 421, 364, 292, 243, 214, 211, 215, 239, 258]
Intensidades de carbono para Itália: [167, 186, 217, 247, 264, 267, 252, 238, 216, 201, 202, 204, 208, 218, 232, 237, 219, 197, 184, 171, 160, 167, 186, 194]
Intensidades de carbono para Portugal: [69, 67, 66, 77, 93, 90, 90, 94, 84, 77, 79, 79, 81, 83, 74, 72, 75, 88, 81, 74, 73, 69, 66, 65]


Ranking geral das empresas:
1. Empresa 3: Score 2.00
2. Empresa 2: Score 0.71
3. Empresa 1: Score 0.62


In [2]:
import numpy as np
import requests

# Função para as intensidades de carbono para uma zona específica no dia anterior
def buscar_intensidades_carbono(zone):
    url = "https://api.electricitymap.org/v3/carbon-intensity/history"
    params = {
        "zone": zone
    }
    headers = {
        "auth-token": "YOUR_API_TOKEN"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        intensidades_carbono = [entry["carbonIntensity"] for entry in data["history"]]
        return intensidades_carbono[-24:]  # Retorna apenas as últimas 24 horas
    else:
        print("Falha ao buscar as intensidades de carbono.")
        return None

# Intensidades de carbono para cada zona no dia anterior
intensidades_carbono_de = buscar_intensidades_carbono("DE")
intensidades_carbono_it = buscar_intensidades_carbono("IT")
intensidades_carbono_pt = buscar_intensidades_carbono("PT")

# Se algum dos valores for None, significa que falhou
if None in (intensidades_carbono_de, intensidades_carbono_it, intensidades_carbono_pt):
    print("Falha ao buscar as intensidades de carbono para uma ou mais zonas.")
    exit()

# Valores das emissões de carbono para cada zona
print("Intensidades de carbono para Alemanha:", intensidades_carbono_de)
print("Intensidades de carbono para Itália:", intensidades_carbono_it)
print("Intensidades de carbono para Portugal:", intensidades_carbono_pt)
print()

# Exemplo de dados de carregamento para três empresas (24 horas)
carregamento_empresa_1 = np.array([5, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4])
carregamento_empresa_2 = np.array([4, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3])
carregamento_empresa_3 = np.array([3, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2])

# Função para calcular as emissões totais diárias
def calcular_emissoes_diarias(carregamento, emissoes):
    emissoes_horarias = carregamento * emissoes
    emissoes_diarias = np.sum(emissoes_horarias)
    return emissoes_diarias

# Função para calcular os cenários de emissões
def calcular_cenarios(carregamento_total, emissoes):
    emissoes_ordenadas_crescente = np.sort(emissoes)
    emissoes_ordenadas_decrescente = np.sort(emissoes)[::-1]
    
    capacidade_por_hora = 10  # Definir capacidade por hora como 10
    horas_necessarias = int(np.ceil(carregamento_total / capacidade_por_hora))
    
    best_case = capacidade_por_hora * emissoes_ordenadas_crescente[:horas_necessarias]
    worst_case = capacidade_por_hora * emissoes_ordenadas_decrescente[:horas_necessarias]
    
    best_case_total = np.sum(best_case)
    worst_case_total = np.sum(worst_case)
    
    return best_case_total, worst_case_total

# Calcular o carregamento total para cada empresa
carregamento_empresa_1_total = np.sum(carregamento_empresa_1)
carregamento_empresa_2_total = np.sum(carregamento_empresa_2)
carregamento_empresa_3_total = np.sum(carregamento_empresa_3)

# Cálculo das emissões reais, best case e worst case para cada empresa
emissoes_empresa_1 = calcular_emissoes_diarias(carregamento_empresa_1, intensidades_carbono_de)
emissoes_empresa_2 = calcular_emissoes_diarias(carregamento_empresa_2, intensidades_carbono_it)
emissoes_empresa_3 = calcular_emissoes_diarias(carregamento_empresa_3, intensidades_carbono_pt)

best_case_1, worst_case_1 = calcular_cenarios(carregamento_empresa_1_total, intensidades_carbono_de)
best_case_2, worst_case_2 = calcular_cenarios(carregamento_empresa_2_total, intensidades_carbono_it)
best_case_3, worst_case_3 = calcular_cenarios(carregamento_empresa_3_total, intensidades_carbono_pt)

# Calcular a distância ao best case e ao worst case
def calcular_distancia_best_worst(emissoes_reais, best_case, worst_case):
    distancia_best = emissoes_reais / best_case
    distancia_worst = emissoes_reais / worst_case
    return distancia_best, distancia_worst

distancia_best_1, distancia_worst_1 = calcular_distancia_best_worst(emissoes_empresa_1, best_case_1, worst_case_1)
distancia_best_2, distancia_worst_2 = calcular_distancia_best_worst(emissoes_empresa_2, best_case_2, worst_case_2)
distancia_best_3, distancia_worst_3 = calcular_distancia_best_worst(emissoes_empresa_3, best_case_3, worst_case_3)

# Normalizar as distâncias ao best case e ao worst case
distancias_best = [distancia_best_1, distancia_best_2, distancia_best_3]
distancias_worst = [distancia_worst_1, distancia_worst_2, distancia_worst_3]

min_dist_best = min(distancias_best)
max_dist_best = max(distancias_best)
min_dist_worst = min(distancias_worst)
max_dist_worst = max(distancias_worst)

norm_dist_best = [(d - min_dist_best) / (max_dist_best - min_dist_best) for d in distancias_best]
norm_dist_worst = [(d - min_dist_worst) / (max_dist_worst - min_dist_worst) for d in distancias_worst]

# Calcular um score geral
scores = [(1 - norm_best) + norm_worst for norm_best, norm_worst in zip(norm_dist_best, norm_dist_worst)]

# Rank das empresas baseado no score geral
empresas = ['Empresa 1', 'Empresa 2', 'Empresa 3']
ranking_geral = sorted(zip(empresas, scores), key=lambda x: x[1])

print("\nRanking geral das empresas:")
for i, (empresa, score) in enumerate(ranking_geral):
    print(f"{i + 1}. {empresa}: Score {score:.2f}")

# Adicionar os valores de distância para referência
for empresa, score, dist_best, dist_worst in zip(empresas, scores, distancias_best, distancias_worst):
    print(f"{empresa}: Score {score:.2f}, Distância ao Best Case: {dist_best:.2f}, Distância ao Worst Case: {dist_worst:.2f}")


Intensidades de carbono para Alemanha: [234, 281, 331, 407, 459, 486, 488, 462, 421, 397, 385, 382, 391, 426, 447, 421, 364, 292, 243, 214, 211, 215, 239, 258]
Intensidades de carbono para Itália: [167, 186, 217, 247, 264, 267, 252, 238, 216, 201, 202, 204, 208, 218, 232, 237, 219, 197, 184, 171, 160, 167, 186, 194]
Intensidades de carbono para Portugal: [69, 67, 66, 77, 93, 90, 90, 94, 84, 77, 79, 79, 81, 83, 74, 72, 75, 88, 81, 74, 73, 69, 66, 65]


Ranking geral das empresas:
1. Empresa 1: Score 0.38
2. Empresa 3: Score 1.00
3. Empresa 2: Score 1.71
Empresa 1: Score 0.38, Distância ao Best Case: 1.16, Distância ao Worst Case: 0.89
Empresa 2: Score 1.71, Distância ao Best Case: 1.08, Distância ao Worst Case: 0.90
Empresa 3: Score 1.00, Distância ao Best Case: 1.05, Distância ao Worst Case: 0.89


In [4]:
import numpy as np
import requests

# Função para buscar as intensidades de carbono para uma zona específica no dia anterior
def buscar_intensidades_carbono(zone):
    url = "https://api.electricitymap.org/v3/carbon-intensity/history"
    params = {
        "zone": zone
    }
    headers = {
        "auth-token": "YOUR_API_TOKEN"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        intensidades_carbono = [entry["carbonIntensity"] for entry in data["history"]]
        return intensidades_carbono[-24:]  # Retorna apenas as últimas 24 horas
    else:
        print("Falha ao buscar as intensidades de carbono.")
        return None

# Buscar as intensidades de carbono para cada zona no dia anterior
intensidades_carbono_de = buscar_intensidades_carbono("DE")
intensidades_carbono_it = buscar_intensidades_carbono("IT")
intensidades_carbono_pt = buscar_intensidades_carbono("PT")

# Se algum dos valores for None, significa que a busca falhou
if None in (intensidades_carbono_de, intensidades_carbono_it, intensidades_carbono_pt):
    print("Falha ao buscar as intensidades de carbono para uma ou mais zonas.")
    exit()

# Agora temos os valores das emissões de carbono para cada zona
print("Intensidades de carbono para Alemanha:", intensidades_carbono_de)
print("Intensidades de carbono para Itália:", intensidades_carbono_it)
print("Intensidades de carbono para Portugal:", intensidades_carbono_pt)

# Exemplo de dados de carregamento para três empresas (24 horas)
carregamento_empresa_1 = np.array([5, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4])
carregamento_empresa_2 = np.array([4, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3])
carregamento_empresa_3 = np.array([3, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2])

# Função para calcular as emissões totais diárias
def calcular_emissoes_diarias(carregamento, emissoes):
    emissoes_horarias = carregamento * emissoes
    emissoes_diarias = np.sum(emissoes_horarias)
    return emissoes_diarias

# Função para calcular os cenários de emissões
def calcular_cenarios(carregamento_total, emissoes):
    emissoes_ordenadas_crescente = np.sort(emissoes)
    emissoes_ordenadas_decrescente = np.sort(emissoes)[::-1]
    
    capacidade_por_hora = 10  # Definindo capacidade por hora como 10
    horas_necessarias = int(np.ceil(carregamento_total / capacidade_por_hora))
    
    best_case = capacidade_por_hora * emissoes_ordenadas_crescente[:horas_necessarias]
    worst_case = capacidade_por_hora * emissoes_ordenadas_decrescente[:horas_necessarias]
    
    best_case_total = np.sum(best_case)
    worst_case_total = np.sum(worst_case)
    
    return best_case_total, worst_case_total

# Calcular o carregamento total para cada empresa
carregamento_empresa_1_total = np.sum(carregamento_empresa_1)
carregamento_empresa_2_total = np.sum(carregamento_empresa_2)
carregamento_empresa_3_total = np.sum(carregamento_empresa_3)

# Cálculo das emissões reais, best case e worst case para cada empresa
emissoes_empresa_1 = calcular_emissoes_diarias(carregamento_empresa_1, intensidades_carbono_de)
emissoes_empresa_2 = calcular_emissoes_diarias(carregamento_empresa_2, intensidades_carbono_it)
emissoes_empresa_3 = calcular_emissoes_diarias(carregamento_empresa_3, intensidades_carbono_pt)

best_case_1, worst_case_1 = calcular_cenarios(carregamento_empresa_1_total, intensidades_carbono_de)
best_case_2, worst_case_2 = calcular_cenarios(carregamento_empresa_2_total, intensidades_carbono_it)
best_case_3, worst_case_3 = calcular_cenarios(carregamento_empresa_3_total, intensidades_carbono_pt)

# Calcular a distância ao best case e ao worst case
def calcular_distancia_best_worst(emissoes_reais, best_case, worst_case):
    distancia_best = (emissoes_reais / best_case - 1) * 100
    distancia_worst = (1 - emissoes_reais / worst_case) * 100
    return distancia_best, distancia_worst

distancia_best_1, distancia_worst_1 = calcular_distancia_best_worst(emissoes_empresa_1, best_case_1, worst_case_1)
distancia_best_2, distancia_worst_2 = calcular_distancia_best_worst(emissoes_empresa_2, best_case_2, worst_case_2)
distancia_best_3, distancia_worst_3 = calcular_distancia_best_worst(emissoes_empresa_3, best_case_3, worst_case_3)

# Resultados
empresas = ['Empresa 1', 'Empresa 2', 'Empresa 3']
distancias_best = [distancia_best_1, distancia_best_2, distancia_best_3]
distancias_worst = [distancia_worst_1, distancia_worst_2, distancia_worst_3]

# Rank das empresas baseado na distância ao best case
ranking_best = sorted(zip(empresas, distancias_best), key=lambda x: x[1])

print("\nRanking das empresas baseado na distância ao best case de emissões de CO2:")
for i, (empresa, distancia) in enumerate(ranking_best):
    print(f"{i + 1}. {empresa}: {abs(distancia):.2f}% {'above' if distancia > 0 else 'below'} the best case scenario")

# Rank das empresas baseado na distância ao worst case
ranking_worst = sorted(zip(empresas, distancias_worst), key=lambda x: x[1])

print("\nRanking das empresas baseado na distância ao worst case de emissões de CO2:")
for i, (empresa, distancia) in enumerate(ranking_worst):
    print(f"{i + 1}. {empresa}: {abs(distancia):.2f}% {'below' if distancia > 0 else 'above'} the worst case scenario")


Intensidades de carbono para Alemanha: [177, 216, 286, 350, 374, 373, 349, 310, 293, 280, 281, 283, 280, 294, 321, 310, 282, 243, 225, 219, 223, 236, 231, 242]
Intensidades de carbono para Itália: [211, 246, 277, 289, 296, 299, 282, 269, 247, 245, 236, 234, 214, 205, 214, 214, 208, 192, 183, 170, 152, 152, 166, 185]
Intensidades de carbono para Portugal: [75, 73, 69, 73, 85, 87, 90, 89, 84, 78, 93, 89, 87, 93, 98, 82, 85, 94, 78, 74, 72, 71, 71, 66]

Ranking das empresas baseado na distância ao best case de emissões de CO2:
1. Empresa 3: 4.79% above the best case scenario
2. Empresa 1: 11.47% above the best case scenario
3. Empresa 2: 12.57% above the best case scenario

Ranking das empresas baseado na distância ao worst case de emissões de CO2:
1. Empresa 1: 7.14% below the worst case scenario
2. Empresa 2: 12.40% below the worst case scenario
3. Empresa 3: 12.43% below the worst case scenario


In [5]:
import numpy as np
import requests

# Função para buscar as intensidades de carbono para uma zona específica no dia anterior
def buscar_intensidades_carbono(zone):
    url = "https://api.electricitymap.org/v3/carbon-intensity/history"
    params = {
        "zone": zone
    }
    headers = {
        "auth-token": "YOUR_API_TOKEN"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        intensidades_carbono = [entry["carbonIntensity"] for entry in data["history"]]
        return intensidades_carbono[-24:]  # Retorna apenas as últimas 24 horas
    else:
        print("Falha ao buscar as intensidades de carbono.")
        return None

# Buscar as intensidades de carbono para cada zona no dia anterior
intensidades_carbono_de = buscar_intensidades_carbono("DE")
intensidades_carbono_it = buscar_intensidades_carbono("IT")
intensidades_carbono_pt = buscar_intensidades_carbono("PT")

# Se algum dos valores for None, significa que a busca falhou
if None in (intensidades_carbono_de, intensidades_carbono_it, intensidades_carbono_pt):
    print("Falha ao buscar as intensidades de carbono para uma ou mais zonas.")
    exit()

# Exemplo de dados de carregamento para três empresas (24 horas)
carregamento_empresa_1 = np.array([5, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4])
carregamento_empresa_2 = np.array([4, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3])
carregamento_empresa_3 = np.array([3, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2])

# Função para calcular as emissões totais diárias
def calcular_emissoes_diarias(carregamento, emissoes):
    emissoes_horarias = carregamento * emissoes
    emissoes_diarias = np.sum(emissoes_horarias)
    return emissoes_diarias

# Função para calcular os cenários de emissões
def calcular_cenarios(carregamento_total, emissoes):
    emissoes_ordenadas_crescente = np.sort(emissoes)
    emissoes_ordenadas_decrescente = np.sort(emissoes)[::-1]
    
    capacidade_por_hora = 10  # Definindo capacidade por hora como 10
    horas_necessarias = int(np.ceil(carregamento_total / capacidade_por_hora))
    
    best_case = capacidade_por_hora * emissoes_ordenadas_crescente[:horas_necessarias]
    worst_case = capacidade_por_hora * emissoes_ordenadas_decrescente[:horas_necessarias]
    
    best_case_total = np.sum(best_case)
    worst_case_total = np.sum(worst_case)
    
    return best_case_total, worst_case_total

# Calcular o carregamento total para cada empresa
carregamento_empresa_1_total = np.sum(carregamento_empresa_1)
carregamento_empresa_2_total = np.sum(carregamento_empresa_2)
carregamento_empresa_3_total = np.sum(carregamento_empresa_3)

# Cálculo das emissões reais, best case e worst case para cada empresa
emissoes_empresa_1 = calcular_emissoes_diarias(carregamento_empresa_1, intensidades_carbono_de)
emissoes_empresa_2 = calcular_emissoes_diarias(carregamento_empresa_2, intensidades_carbono_it)
emissoes_empresa_3 = calcular_emissoes_diarias(carregamento_empresa_3, intensidades_carbono_pt)

best_case_1, worst_case_1 = calcular_cenarios(carregamento_empresa_1_total, intensidades_carbono_de)
best_case_2, worst_case_2 = calcular_cenarios(carregamento_empresa_2_total, intensidades_carbono_it)
best_case_3, worst_case_3 = calcular_cenarios(carregamento_empresa_3_total, intensidades_carbono_pt)

# Calcular a distância ao best case e ao worst case
def calcular_distancia_best_worst(emissoes_reais, best_case, worst_case):
    distancia_best = (emissoes_reais / best_case - 1) * 100
    distancia_worst = (1 - emissoes_reais / worst_case) * 100
    return distancia_best, distancia_worst

distancia_best_1, distancia_worst_1 = calcular_distancia_best_worst(emissoes_empresa_1, best_case_1, worst_case_1)
distancia_best_2, distancia_worst_2 = calcular_distancia_best_worst(emissoes_empresa_2, best_case_2, worst_case_2)
distancia_best_3, distancia_worst_3 = calcular_distancia_best_worst(emissoes_empresa_3, best_case_3, worst_case_3)

# Calcular a média simples das distâncias
def calcular_media_simples(distancia_best, distancia_worst):
    return (distancia_best + distancia_worst) / 2

media_simples_1 = calcular_media_simples(distancia_best_1, distancia_worst_1)
media_simples_2 = calcular_media_simples(distancia_best_2, distancia_worst_2)
media_simples_3 = calcular_media_simples(distancia_best_3, distancia_worst_3)

# Resultados
empresas = ['Empresa 1', 'Empresa 2', 'Empresa 3']
medias_simples = [media_simples_1, media_simples_2, media_simples_3]

# Rank das empresas baseado na média simples das distâncias
ranking_final = sorted(zip(empresas, medias_simples), key=lambda x: x[1])

print("\nRanking das empresas baseado na média simples das distâncias ao best case e worst case de emissões de CO2:")
for i, (empresa, media) in enumerate(ranking_final):
    print(f"{i + 1}. {empresa}: média de {media:.2f}%")

# Exibir as distâncias individuais ao best case e worst case
print("\nDistâncias ao best case e worst case de emissões de CO2:")
for i, empresa in enumerate(empresas):
    if empresa == 'Empresa 1':
        print(f"{empresa}: {distancia_best_1:.2f}% above the best case, {distancia_worst_1:.2f}% below the worst case")
    elif empresa == 'Empresa 2':
        print(f"{empresa}: {distancia_best_2:.2f}% above the best case, {distancia_worst_2:.2f}% below the worst case")
    elif empresa == 'Empresa 3':
        print(f"{empresa}: {distancia_best_3:.2f}% above the best case, {distancia_worst_3:.2f}% below the worst case")



Ranking das empresas baseado na média simples das distâncias ao best case e worst case de emissões de CO2:
1. Empresa 3: média de 8.61%
2. Empresa 1: média de 9.31%
3. Empresa 2: média de 12.49%

Distâncias ao best case e worst case de emissões de CO2:
Empresa 1: 11.47% above the best case, 7.14% below the worst case
Empresa 2: 12.57% above the best case, 12.40% below the worst case
Empresa 3: 4.79% above the best case, 12.43% below the worst case
